In [2]:
import pandas as pd
# Load dataset
file_path = "Stock_Features_Complete.xlsx"
df_long = pd.read_excel(file_path)

# Ensure Date is in datetime format
df_long["Date"] = pd.to_datetime(df_long["Date"])

# Sort data by Ticker & Date
df_long = df_long.sort_values(by=["Ticker", "Date"]).reset_index(drop=True)
print(df_long.head())

        Date       Ticker  Closing_Price  Simple_Return  Log_Return  \
0 2020-03-31  A UN Equity          71.62            NaN         NaN   
1 2020-04-30  A UN Equity          76.66       0.070371    0.068006   
2 2020-05-29  A UN Equity          88.14       0.149752    0.139546   
3 2020-06-30  A UN Equity          88.37       0.002609    0.002606   
4 2020-07-31  A UN Equity          96.33       0.090076    0.086247   

   Rolling_20D_Volatility  SMA_50  SMA_200     EMA_12     EMA_26      MACD  \
0                     NaN     NaN      NaN  71.620000  71.620000  0.000000   
1                     NaN     NaN      NaN  72.395385  71.993333  0.402051   
2                     NaN     NaN      NaN  74.817633  73.189383  1.628250   
3                     NaN     NaN      NaN  76.902613  74.313873  2.588740   
4                     NaN  84.224   84.224  79.891441  75.944697  3.946744   

     RSI  SMA_20  Upper_BB  Lower_BB  
0    NaN     NaN       NaN       NaN  
1    NaN     NaN       NaN

LSTM-Based Stock Price Prediction and Portfolio Performance Comparison
1. Prepare Data for Each Ticker Separately
Data Extraction: Begin by isolating each stock’s historical data from the combined dataset. For each ticker, collect its time-indexed features (e.g. Date, Open, High, Low, Close, Volume, etc.). Ensure the data is sorted in chronological order (oldest to newest) for proper time-series processing. This per-ticker separation means you will train a dedicated LSTM on each stock’s own data (no mixing tickers in one model), allowing the model to learn that stock’s patterns and idiosyncrasies. Feature Engineering: Enrich each ticker’s dataset with time-series features that could improve predictive power. Common features include returns and rolling statistics. For example, you can compute daily or monthly returns, multi-period returns (e.g. 20-day return), and rolling volatility over a window​
FILE-J3DLCEUFNY6EQJKMHWWRNY
. Technical indicators or normalized versions of these features (e.g. z-scores over a rolling year​
FILE-J3DLCEUFNY6EQJKMHWWRNY
) can also be added. Make sure any feature is computed using past data only (no future leakage). By structuring the data with relevant features, the LSTM can capture market dynamics beyond raw price. Each ticker’s feature set might look like: [Open, High, Low, Close, Volume, Returns, Volatility, etc.] for each date.
2. Structure Data in Time-Series Sequence Format
Sliding Window Sequences: Organize each ticker’s data into input/output sequences for the LSTM. Decide on a lookback window length (number of past time steps the model will use for prediction) and a forecast horizon (how many future steps to predict). For instance, with daily data you might use the past 252 trading days (≈1 year) to predict the next 22 days (≈1 month)​
FILE-J3DLCEUFNY6EQJKMHWWRNY
​
FILE-J3DLCEUFNY6EQJKMHWWRNY
. If using monthly data, you could use the past 12 months to predict the next 1 or 2 months, etc. This creates a sequence-to-sequence mapping: the input $X$ is a sequence of length lookback and the target $yis the subsequenthorizon` values. No Data Leakage: Make sure to maintain temporal order in each sequence. Generate sequences by moving a sliding window forward in time. For example, the first sequence for a stock could be January–December (inputs) predicting January of next year (output), the next sequence would shift one step forward (or by the horizon length) and so on. Typically a sliding window with step=1 (overlapping windows) is used to maximize training samples. The provided methodology used a “stateful” approach with no overlap (window jumps by the horizon each time) to avoid reusing the same time points in multiple sequences​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. You may choose overlapping windows (which is more common and usually fine) or non-overlapping windows as in the original code – either way, ensure each sequence is strictly chronological. Each LSTM input will thus be a 3D tensor of shape (samples, lookback, features) and the corresponding output can be a single value or a sequence of length horizon for multi-step forecasts.
3. Train-Test Split with Time Order
Chronological Split: When dividing data into training and testing sets, do not shuffle the data – preserve the time order. Use the earliest portion of each ticker’s data for training and the most recent portion for testing. For example, you might take the first ~80% of the time steps as the training set and the last 20% as the test set (as done in the provided script)​
FILE-J3DLCEUFNY6EQJKMHWWRNY
. This ensures the model is always trained on past data and tested on future data, mirroring real-world forecasting. If the dataset is large, you can also set aside a slice of the training period as a validation set for tuning (e.g. split the training data into train vs. validation by time as well). Avoid random sampling in splitting, since that would intermingle past and future data and invalidate the time-series nature. The key is that all model tuning and training happens using data from before a certain cutoff date, and evaluation is on data after that date. Scaling: Before training, apply feature scaling fit only on the training data, then transform both train and test. The common approach (also used in the scripts) is Min-Max scaling to [0,1] range for each feature​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. This prevents features with larger magnitudes (e.g. prices or volume) from dominating others and helps the LSTM converge. Remember to save the scaler fitted on training data and use it to inverse-transform predictions later so you can evaluate results in real dollar terms. Scaling is done per ticker’s feature set. (If your features have very different distributions, alternative scaling like standardization could be considered, but Min-Max is straightforward and was shown to work in the prior approach​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
.)
4. LSTM Model Architecture for Each Ticker
Network Design: Build a separate LSTM-based neural network for each stock’s data. A proven architecture is a stacked LSTM: for example, two LSTM layers followed by a Dense output layer. The prior project found that using two LSTM layers yielded better performance than a single layer​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
 (despite some literature suggesting a single layer might suffice). For each ticker’s model, you can start with one LSTM layer and then experiment with adding a second. In a two-layer setup, the first LSTM layer returns sequences (passing its output sequence to the next LSTM), and the second LSTM layer can either return another sequence or just the final state depending on whether you want further layers. In practice, a common pattern is:
Layer 1: LSTM(units=X, return_sequences=True, input_shape=(lookback, num_features))
Layer 2: LSTM(units=Y, return_sequences=False) (or True if you plan to add more LSTM layers)
Output Layer: Dense(units=horizon, activation='linear') for regression output of future price(s).
Here units X, Y are hyperparameters (number of neurons) that can be tuned. The provided code, for instance, used a very large LSTM with 800 units in the first layer and 600 in the second for a multi-stock model​
FILE-J3DLCEUFNY6EQJKMHWWRNY
​
FILE-J3DLCEUFNY6EQJKMHWWRNY
. For individual tickers (and likely fewer input features), you can use a smaller network (e.g. 50–100 neurons per layer as a starting point) to avoid overfitting. Include a Dropout layer (e.g. 20%) after each LSTM if overfitting is a concern, especially if your training data per ticker is not very large. The Dense output layer should have 1 unit if you predict a single step ahead, or horizon units if predicting multiple future steps at once. Use a linear activation on the output for price regression (the example code used a final sigmoid after a relu dense, but ultimately they inverse-scaled the results, achieving essentially regression output​
FILE-J3DLCEUFNY6EQJKMHWWRNY
). Compilation: Compile the model with a regression-friendly loss function like mean squared error (MSE), and an optimizer such as Adam. We can monitor metrics like MAE or MSE (accuracy metric isn’t very informative for regression, even though the script included it). For example: model.compile(loss='mean_squared_error', optimizer='adam'). This setup will train the LSTM to minimize the MSE between predicted and actual prices.
5. Training the LSTM Models
Model Training: Train each ticker’s LSTM on its training set sequences and evaluate on its test set. Use the earlier defined train/validation split for hyperparameter tuning: for instance, train for a fixed number of epochs and observe validation loss to prevent overfitting. It’s often helpful to use early stopping on validation loss if available (patience of a few epochs) to stop training when performance stops improving. (The provided code commented out early stopping because it underfit with their configuration​
FILE-J3DLCEUFNY6EQJKMHWWRNY
, but in general it’s a good practice if you have enough data.) Use a batch size that balances learning stability and speed (common values are 16–64 for time series; the example used 256 batch size​
FILE-J3DLCEUFNY6EQJKMHWWRNY
 likely due to a large combined dataset). Ensure you shuffle batch order during training (shuffle=False in the .fit() call was used​
FILE-J3DLCEUFNY6EQJKMHWWRNY
 since sequences are already time-ordered; keeping it False is fine and ensures each epoch sees sequences in chronological order). Hyperparameter Optimization: To optimize the model architecture for accuracy, consider tuning hyperparameters such as the number of LSTM layers, number of units, dropout rate, lookback window size, and learning rate. The methodology suggests that these were initially chosen by heuristics, and further systematic tuning (e.g. via cross-validation) could improve performance​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. For example, you might perform a grid search or manual experimentation to see if a single LSTM layer with more units performs better or worse than two layers with fewer units, or adjust the lookback length to capture enough history without overfitting. Since time-series cross-validation can be tricky, one approach is a rolling walk-forward validation where you train on an expanding window and test on the next chunk, iterating through the training period. The goal of tuning is to minimize validation error and ultimately produce more accurate forecasts on the test set. Keep in mind the bias-variance tradeoff: a very complex model may overfit, while a too-simple model may underfit. Use the model that gives the best validation performance and generalizes well to the test data.
6. Future Price Forecasting for Each Ticker
Generating Forecasts: Once trained, use each model to predict future prices for its respective stock. If your model outputs a single-step forecast, you can predict one step ahead in a loop (feeding the model with an updated window that includes the newly predicted value to get the next prediction, and so on) to produce a multi-step forecast. Alternatively, if the model was designed to predict a fixed horizon (e.g. 22 days) in one go (as in the provided approach), you can get the full multi-day forecast at once by feeding in the latest lookback window from the end of the training data​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. For example, using daily data with horizon=22, take the last 252 trading days of known data for a stock, scale it, reshape to (1,252,features), and use model.predict() to get a 22-day forward price trajectory. For monthly data, a model with horizon=1 might be more realistic (predict next month), but you could also attempt 3-6 months horizon if sequence length and data volume permit. Inverse-Transforming Predictions: The predicted values will be in the scaled form (e.g. 0–1 range if MinMax scaler was used). Apply the inverse scaling to convert each prediction back to the original price scale​
FILE-J3DLCEUFNY6EQJKMHWWRNY
​
FILE-J3DLCEUFNY6EQJKMHWWRNY
. It’s important to invert the scaling using the same scaler that was fit on the training data of that ticker to get meaningful price forecasts in dollars. Once you have actual price forecasts for each future time step, you can also compute forecasted returns from them. The prior methodology computed log returns from predicted prices​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
 – log returns are often preferred because they are additive over time and more normally distributed, which simplifies portfolio optimization calculations. You can do the same: for each predicted price $P_{t}$, compute log return as $\ln(P_{t}/P_{t-1})$. These forecasted returns will be used as inputs to the portfolio optimization. (If using a 1-step ahead model, you’ll recompute predictions and returns at each rebalance date; if using a multi-step horizon model, you get a block of returns for the horizon.)
7. Portfolio Optimization Using Forecasts
Constructing the Portfolio: With future return forecasts for each asset, we next determine how to allocate the portfolio across the tickers. We want to maximize return and manage risk based on predictions. A common approach is mean-variance optimization (Markowitz) with constraints, often maximizing the Sharpe ratio (return/risk). In this project’s context, an optimization problem was formulated to maximize a combination of expected return and a penalty for risk and complexity​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. Specifically, they used a custom objective that subtracts a term for portfolio variance and adds an L1 regularization term to encourage sparse asset weights​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. In practice, this means the optimizer will prefer to concentrate on a few stocks that it’s most confident about (sparsity) rather than spreading thinly across many stocks. The constraints typically include: weights sum to 1 (100% of capital invested)​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
, and possibly limit on short selling (e.g. weights ≥ 0 if no short positions). You may also set a target expected return or allow the optimizer to free choose weights to maximize Sharpe. Use the predicted returns as the estimate of expected return for each stock, and you can use a covariance matrix of returns (perhaps estimated from recent historical data or the variability of the predictions) as the risk measure. Solving this optimization (e.g. with quadratic programming or coordinate descent for L1) gives the weight for each asset in the portfolio for the next period. Rebalancing Over Time: In alignment with the methodology, you would form a new portfolio at the start of each forecast horizon and hold it until the horizon ends​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. For example, if forecasting 1 month ahead, you’d allocate weights based on those forecasts, hold for 1 month, then at month’s end, get new predictions and rebalance for the next month. In the provided approach, they forecasted 22 trading days ahead and rebalanced every 22 days (approximately monthly)​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. Over the test period, this creates a series of portfolio decisions. Track the performance by applying the actual realized returns of each stock (not the predicted returns) to the chosen weights in each period. This will give you the portfolio’s value over time as you rebalance. Essentially, you are simulating an investor who uses the LSTM model predictions to inform portfolio choices on a rolling basis.
8. Comparing Portfolio Performance Against S&P 500
Benchmarking: Finally, evaluate how the LSTM-driven portfolio performs relative to the S&P 500 index (the benchmark). Compute common performance metrics for the portfolio versus the benchmark over the same test period. Key metrics include: total return, annualized return, annualized volatility, and Sharpe ratio. For example, if you start both the portfolio and the S&P 500 at an initial value of 100, you can calculate the cumulative return over time to see growth of wealth​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. Plotting the equity curve of the portfolio vs. the S&P 500 provides a visual comparison of performance. In the referenced project’s results, the LSTM-based strategy’s equity curve showed higher growth than baseline models for most of the period​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. You would expect a successful model to outperform the S&P 500 if it’s capturing predictive signals effectively. Calculate the Sharpe ratio for both the portfolio and S&P: even if raw returns are higher, it’s important to check if they were achieved with reasonable risk. The S&P 500’s performance can be obtained from its price data (or an ETF like SPY) over the test horizon – compute its returns and the same metrics for comparison. Discussion: Interpret the results. If the LSTM portfolio outperforms the S&P 500 (higher cumulative return and Sharpe), this indicates value added by the model’s predictions. If not, it suggests the model may not be capturing enough signal or the market is hard to beat. The prior research noted that what matters is correctly predicting the direction of returns rather than exact prices​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
 – an LSTM can still help if it guides you toward the right trades even if MSE on prices is not the lowest. They found the LSTM-based portfolio was the second-best model (outperforming a linear model) in terms of equity gain over 5 years​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
. In your case, a direct S&P 500 comparison will show whether the active, model-driven portfolio yields better risk-adjusted returns than a passive index investment. This validation against a benchmark is crucial for assessing the practical value of the predictive model in a portfolio context. Conclusion: By training individual LSTM models per ticker with properly structured time-series data and using their forecasts to inform a portfolio allocation, you align with the methodology of the provided scripts while adapting to the new dataset. The approach involves careful sequence preparation, rigorous training on past data, multi-step price forecasting, and an optimization step to turn forecasts into investment decisions. Throughout, maintaining the temporal order of data and evaluating against a market benchmark (S&P 500) ensures the strategy is realistic and its performance is meaningfully assessed. Following these steps will yield a portfolio whose performance you can compare to the S&P 500 to determine if the LSTM-based strategy offers an improvement over a simple index investment. Sources: The methodology and choices above are based on the approach from the provided scripts and project documentation​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
​
FILE-Q7UFKPZFYFH4WM5W9DHBK6
, which implemented a double-layer LSTM for stock price prediction and used the predictions in a portfolio optimization framework with rebalancing. This ensures our implementation is consistent with that prior work while tailored to the specific data at hand.

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load dataset
file_path = "Stock_Features_Complete.xlsx"
df = pd.read_excel(file_path)

# Convert Date column to datetime
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(by=["Ticker", "Date"]).reset_index(drop=True)

# Feature columns (excluding Date and Ticker)
feature_columns = df.columns[2:]

# Normalize each stock separately
scalers = {}
df_scaled = df.copy()

for ticker in df["Ticker"].unique():
    stock_data = df[df["Ticker"] == ticker][feature_columns]
    scaler = MinMaxScaler()
    df_scaled.loc[df["Ticker"] == ticker, feature_columns] = scaler.fit_transform(stock_data)
    scalers[ticker] = scaler  # Save scaler for inverse transformation

# Convert to time series format for LSTM
def create_sequences(data, seq_length=10):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]  # Last `seq_length` time steps
        target = data[i+seq_length, 0]  # Predict stock price (1st column)
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)


In [8]:
sequence_length = 10  # Past 10 time steps for prediction

train_data = {}
test_data = {}

for ticker in df["Ticker"].unique():
    stock_data = df_scaled[df_scaled["Ticker"] == ticker][feature_columns].values
    X, y = create_sequences(stock_data, sequence_length)
    
    # Time-based split (80-20)
    split_idx = int(0.8 * len(X))
    train_data[ticker] = (X[:split_idx], y[:split_idx])
    test_data[ticker] = (X[split_idx:], y[split_idx:])

print("\n✅ Data Preprocessing Complete.")



✅ Data Preprocessing Complete.


In [9]:
# Train an LSTM model for each ticker
models = {}

for ticker in df["Ticker"].unique():
    X_train, y_train = train_data[ticker]
    
    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        LSTM(50, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(25, activation='relu'),
        Dense(1)  # Predicting stock price
    ])

    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1)

    models[ticker] = model  # Save trained model

print("\n✅ All LSTM Models Trained Successfully.")


Epoch 1/30


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: nan 
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: nan
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: nan
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: nan
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: nan
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: nan
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan
Epoch 13/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 14/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan
Epoch 15/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 16/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 17/30
2/2 ━━━━━━━━━━━